# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
# import the requests library (1 line)
import requests as rq

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
url ="https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = url + '/status'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req= rq.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code==200:
       print("Server is up and running!")
else:
    print(f"Server returned status code {req.status_code}. Something might be wrong.")


Server is up and running!


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [11]:
# Set the countries_url variable (1 line)
url_countries="https://country-leaders.onrender.com/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = rq.get(url_countries)
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print (req.status_code, countries)

403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [82]:
# Set the cookie_url variable (1 line)
url_cookie=url + "/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies= rq.get(url_cookie).cookies
print(cookies.get_dict())


{'user_cookie': '1a8a2628-2392-4eac-9be5-6a88bcc24f2a'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [83]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = rq.get(url + "/countries",cookies=cookies).json()
# display the countries variable (1 line)
print(countries)

['fr', 'be', 'ma', 'us', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [84]:
# Set the leaders_url variable (1 line)
leaders_url = url +"/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = rq.get(leaders_url, cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [63]:
import random
# query the /leaders endpoint using cookies and parameters (take any country in countries)
#choose randomly a country
random_country = random.choice(countries)
# assign the output to the leaders variable (1 line)
leaders = rq.get(leaders_url,cookies=cookies, params={"country": random_country}).json()
# display the leaders variable (1 line)
print (leaders)

[{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leaders_per_country ={}
for country in countries:
    leaders = rq.get(leaders_url,cookies=cookies, params={"country": country}).json()
    leaders_per_country[country]=leaders  # Add leader list to countries
print (leaders_per_country)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

In [85]:
# or 1 line
leaders_per_country[country]= {country:rq.get(leaders_url,cookies=cookies, params={"country": country}).json() for country in countries}
print (leaders_per_country)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [96]:
# < 15 lines
def get_leaders()-> dict:
    url ="https://country-leaders.onrender.com"
    # get the status
    req= rq.get(url+'/status')  
    if req.status_code==200:
       print("Server is up and running!")
    else:
        print(f"Server returned status code {req.status_code}. Something might be wrong.")    
    # get the cookies
    cookies= rq.get(url_cookie).cookies
    # get the countries
    countries = rq.get(url + "/countries",cookies=cookies).json()
    return {country: rq.get(url + "/leaders", cookies=cookies, params={"country": country}).json() for country in countries}




Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [97]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

Server is up and running!
{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_bir

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [147]:
# 3 lines
# Pick a random country, then a random leader from that country
random_country = random.choice(list(leaders_per_country.keys()))
random_leader = random.choice(leaders_per_country[random_country])
text = rq.get(random_leader['wikipedia_url']).text
print(text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="ar" dir="rtl">
<head>
<meta charset="UTF-8">
<title>محمد الخامس بن يوسف - ويكيبيديا</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [148]:
# 3 lines
from bs4 import BeautifulSoup as bs
soup = bs(rq.get(random_leader['wikipedia_url']).content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="rtl" lang="ar">
 <head>
  <meta charset="utf-8"/>
  <title>
   محمد الخامس بن يوسف - ويكيبيديا
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-con

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [149]:
# 2 lines
paragraphs = soup.find_all('p')
print (paragraphs[0])

<p><b>محمد الخامس</b> بن <a href="/wiki/%D9%8A%D9%88%D8%B3%D9%81_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86" title="يوسف بن الحسن">يوسف</a> بن <a href="/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%A3%D9%88%D9%84_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF" title="الحسن الأول بن محمد">الحسن</a> بن <a href="/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B1%D8%A7%D8%A8%D8%B9_%D8%A8%D9%86_%D8%B9%D8%A8%D8%AF_%D8%A7%D9%84%D8%B1%D8%AD%D9%85%D9%86" title="محمد الرابع بن عبد الرحمن">محمد</a> بن <a href="/wiki/%D8%A3%D8%A8%D9%88_%D8%A7%D9%84%D9%81%D8%B6%D9%84_%D8%B9%D8%A8%D8%AF_%D8%A7%D9%84%D8%B1%D8%AD%D9%85%D9%86_%D8%A8%D9%86_%D9%87%D8%B4%D8%A7%D9%85" title="أبو الفضل عبد الرحمن بن هشام">عبد الرحمن</a> بن <a href="/wiki/%D9%87%D8%B4%D8%A7%D9%85_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF" title="هشام بن محمد">هشام</a> بن <a href="/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AB%D8%A7%D9%84%D8%AB_%D8%A8%D9%86_%D8%B9%D8%A8%D8%AF_%D8%A7%D9%84%D9%84%D9%87" title="محمد الثالث بن عبد الله">محمد</a> بن 

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [151]:
# <10 lines

for p in soup.select("#mw-content-text p"): ##  All paragraphs (<p>) in div with id= mw-content-text , soup.select() is a css selector
    text = p.get_text(strip=True)
    if len(text) > 80 and "." in text and not text.startswith(("[", "(", "Coordinates")):
        first_paragraph= text
        break
    else:
        first_paragraph="No suitable paragraph found."
print(first_paragraph)

محمد الخامسبنيوسفبنالحسنبنمحمدبنعبد الرحمنبنهشامبنمحمدبنعبد اللهبنإسماعيلبنالشريف بن عليالعلوي وُلد (1327 هـ/10 أغسطس1909مبالقصر السلطانيبفاس)[1]وتوفي (1381 هـ/26 فبراير1961مبالرباط) خَلَف والده السلطانمولاي يوسفالذي توفي بُكرة يوم الخميس22 جمادى الأولىسنة1346 هـموافق17 نوفمبرسنة1927م[2]فبويع ابنه سيدي محمد سلطانا للمغرب في اليوم الموالي بعد صلاة الجمعة23 جمادى الأولىسنة1346 هـموافق18 نوفمبرسنة1927مفي القصر السلطانيبفاس[3]ولم يزل سلطان المغرب إلى سنة1957م، قضى منها المنفى بين (1953-1955)، ثم اتخذ لقب الملك سنة1957مولم يزل ملكا إلى وفاته سنة1961م، ساند السلطانمحمد الخامسنضالاتالحركة الوطنية المغربيةالمطالبة بتحقيقالاستقلال، الشيء الذي دفعه إلى الاصطدامبسلطات الحماية. وكانت النتيجة قيام سلطات الحماية بنفيه إلىمدغشقر. وعلى إثر ذلك اندلعت مظاهرات مطالبة بعودته إلى وطنه. وأمام اشتداد حدة المظاهرات، قبلت السلطات الفرنسية بإرجاع السلطان إلى عرشه يوم16 نوفمبر1955. وبعد بضعة شهور تم إعلاناستقلالالمغرب. كان الملك محمد الخامس يكنى: أبا عبد الله.[1][4][5]


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [156]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
def extract_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    soup = bs(rq.get(wikipedia_url).content, 'html.parser')
    for p in soup.select("#mw-content-text p"): #  All paragraphs (<p>) in div with id= mw-content-text 
        text = p.get_text(strip=True)
        if len(text) > 80 and "." in text and not text.startswith(("[", "(", "Coordinates")):
           return text  # return the first paragraph finded 
    return "No suitable paragraph found."  # if there is no suitable paragraph

In [ ]:
# Test: 3 lines
#choose a country and a leader randomly
random_country = random.choice(list(leaders_per_country.keys()))
random_leader = random.choice(leaders_per_country[random_country])
#pass the wikipedia url of the leader and show the result
print(extract_first_paragraph(random_leader["wikipedia_url"]))


الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي(9 يوليو 1929 – 23 يوليو 1999)[13]هو ثاني ملوكالمملكة المغربيةبعد الإستقلال، والملك الثاني والعشرون للمغرب منسلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملكمحمد الخامسفي 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلىالحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [168]:
# 3 lines
import re
 # Delete the references [1], [note 2], etc.
text = extract_first_paragraph(random_leader["wikipedia_url"])
text = re.sub(r'\[[^\]]*\]', '', text)
print (text)


الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي(9 يوليو 1929 – 23 يوليو 1999)هو ثاني ملوكالمملكة المغربيةبعد الإستقلال، والملك الثاني والعشرون للمغرب منسلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملكمحمد الخامسفي 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلىالحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [173]:
# 10 lines
def extract_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    soup = bs(rq.get(wikipedia_url).content, 'html.parser')
    for p in soup.select("#mw-content-text p"): #  All paragraphs (<p>) in div with id= mw-content-text 
        text = p.get_text(strip=True)
        if len(text) > 80 and "." in text and not text.startswith(("[", "(", "Coordinates")):
           text = re.sub(r'\[[^\]]*\]', '', text)  # Delete the references [1], [note 2], etc.
           return text  # return the first paragraph finded 
    return "No suitable paragraph found."  # if there is no suitable paragraph


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    soup = bs(rq.get(wikipedia_url).content, 'html.parser')
    for p in soup.select("#mw-content-text p"): #  All paragraphs (<p>) in div with id= mw-content-text 
        text = p.get_text(strip=True)
        if len(text) > 80 and "." in text and not text.startswith(("[", "(", "Coordinates")):
           clean_text = re.sub(r'\[[^\]]*\]', '', text)  # Delete the references [1], [note 2], etc.
           clean_text = re.sub(r'\([^)]*[/ˈˌ][^)]*\)', '', text) # Delete the  prononciations between ()contenant | or ... etc
           clean_text = re.sub(r'\(([^)]{1,25})\)', '', text) # Delete the other courts parentheses (definitions, years, etc.)
           clean_text = re.sub(r'\s+', ' ', text) # Clean the over spaces
           return clean_text # return the first paragraph finded 
    return "No suitable paragraph found."  # if there is no suitable paragraph


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [199]:
# < 20 lines
def get_leaders()-> dict:
    url ="https://country-leaders.onrender.com"
    # get the status
    req= rq.get(url+'/status')  
    if req.status_code==200:
       print("Server is up and running!")
    else:
        print(f"Server returned status code {req.status_code}. Something might be wrong.")    
    # get the cookies
    cookies= rq.get(url_cookie).cookies
    # get the countries
    countries = rq.get(url + "/countries",cookies=cookies).json()
    #leaders= {country: rq.get(url + "/leaders", cookies=cookies, params={"country": country}).json() for country in countries}
    # Get leaders and enrich with first paragraph
    all_leaders = {}
    for country in countries:
        response = rq.get(url + "/leaders", cookies=cookies, params={"country": country}).json()
        leaders = response if isinstance(response, list) else response.get("leaders", [])

        for leader in leaders:
            wikipedia_url = leader.get("wikipedia_url")
            if wikipedia_url:
                leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

        all_leaders[country] = leaders

    return all_leaders

In [200]:
# Check the output of your function (2 lines)
from pprint import pprint 
"""The pprint() (pretty print) function formats nested dictionaries better for clear reading, 
unlike print() which quickly becomes unreadable if your object contains a lot of data 
(like here with several countries and leaders).
"""
leaders = get_leaders()

pprint(leaders)

Server is up and running!
{'be': [],
 'fr': [],
 'ma': [{'birth_date': '1963-08-21',
         'death_date': None,
         'end_mandate': None,
         'first_name': 'Mohammed',
         'first_paragraph': 'مُحمد السادس بن الحسن الثاني العلوي هو ملك '
                            'المملكة المغربية منذ عام 1999 والملك الثالث '
                            'والعشرون للمغرب منسلالة العلويين الفيلاليين، تولى '
                            'الحكم خلفًا لوالده الملكالحسن الثانيبعد وفاته، '
                            'وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ '
                            'الموافق 23 يوليو 1999 بالقصر الملكيبالرباط.',
         'id': 'Q57553',
         'last_name': 'None',
         'place_of_birth': 'Rabat',
         'start_mandate': '1999-07-23',
         'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86'},
        {'birth_date': '1929-07-09',
         'death_date': '

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [203]:
# < 25 lines
def get_leaders()-> dict:
    url ="https://country-leaders.onrender.com"
    # get the status
    req= rq.get(url+'/status')  
    if req.status_code==200:
       print("Server is up and running!")
    else:
        print(f"Server returned status code {req.status_code}. Something might be wrong.")    
    # get the cookies
    cookies= rq.get(url_cookie).cookies
    # get the countries
    countries = rq.get(url + "/countries",cookies=cookies).json()
    #leaders= {country: rq.get(url + "/leaders", cookies=cookies, params={"country": country}).json() for country in countries}
    # Get leaders and enrich with first paragraph
    all_leaders = {}
    for country in countries:
        try:
            # First attempt
            response = rq.get(url + "/leaders", cookies=cookies, params={"country": country})
            
            # If cookie expired: get new cookie and retry once
            if response.status_code in [401, 403]:
                print(f"Cookie expired for {country}, fetching new cookie...")
                cookies = rq.get(url + "/cookie").cookies
                response = rq.get(url + "/leaders", cookies=cookies, params={"country": country})
            
            # If still fails, log and skip
            if response.status_code != 200:
                print(f"Failed to get leaders for {country}: {response.status_code}")
                all_leaders[country] = []
                continue

            leaders = response.json() if isinstance(response.json(), list) else response.json().get("leaders", [])

            for leader in leaders:
                wikipedia_url = leader.get("wikipedia_url")
                if wikipedia_url:
                    leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

            all_leaders[country] = leaders

        except Exception as e:
            print(f"Error for {country}: {e}")
            all_leaders[country] = []

    
        all_leaders[country] = leaders
   
    return all_leaders


Check the output of your function again.

In [204]:
# Check the output of your function (1 line)
leaders = get_leaders()

pprint(leaders)

Server is up and running!
Cookie expired for ma, fetching new cookie...
Cookie expired for ru, fetching new cookie...
{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt is '
                            'eenBelgischpoliticusvoor deOpen Vlaamse Liberalen '
                            'en Democraten. Hij waspremier van Belgiëvan 12 '
                            'juli 1999 tot 20 maart 2008 in drie regeringen. '
                            'Hij beëindigde zijn actieve politieke carrière in '
                            'hetEuropees Parlement, waar hij van 2009 tot 2019 '
                            'fractieleider van deAlliantie van Liberalen en '
                            'Democraten voor Europa was.',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
         'place_of_birth': 'Dendermonde',
         'start_mandate': '1999-07-1

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [205]:
# < 20 lines
""" using a single requests.Session is the best practice for multiple requests to the same site (Wikipedia here). 
It reuses TCP connections, which is faster and more robust."""
def get_first_paragraph(wikipedia_url,session):
    # Use the passed session instead of rq.get()
    soup = bs(session.get(wikipedia_url).content, 'html.parser')
    for p in soup.select("#mw-content-text p"): #  All paragraphs (<p>) in div with id= mw-content-text 
        text = p.get_text(strip=True)
        if len(text) > 80 and "." in text and not text.startswith(("[", "(", "Coordinates")):
           clean_text = re.sub(r'\[[^\]]*\]', '', text)  # Delete the references [1], [note 2], etc.
           clean_text = re.sub(r'\([^)]*[/ˈˌ][^)]*\)', '', text) # Delete the  prononciations between ()contenant | or ... etc
           clean_text = re.sub(r'\(([^)]{1,25})\)', '', text) # Delete the other courts parentheses (definitions, years, etc.)
           clean_text = re.sub(r'\s+', ' ', text) # Clean the over spaces
           return clean_text # return the first paragraph finded 
    return "No suitable paragraph found."  # if there is no suitable paragraph


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [208]:
# <25 lines
def get_leaders()-> dict:
    url ="https://country-leaders.onrender.com"
    # get the status
    req= rq.get(url+'/status')  
    if req.status_code==200:
       print("Server is up and running!")
    else:
        print(f"Server returned status code {req.status_code}. Something might be wrong.")    
    # get the cookies
    cookies= rq.get(url_cookie).cookies
    # get the countries
    countries = rq.get(url + "/countries",cookies=cookies).json()
    #leaders= {country: rq.get(url + "/leaders", cookies=cookies, params={"country": country}).json() for country in countries}
    # Get leaders and enrich with first paragraph
    all_leaders = {}
    
    # Create a single session for Wikipedia scraping
    with rq.Session() as wiki_session:
        for country in countries:
            try:
                response = rq.get(url + "/leaders", cookies=cookies, params={"country": country})

                if response.status_code in [401, 403]:
                    print(f"Cookie expired for {country}, fetching new cookie...")
                    cookies = rq.get(url + "/cookie").cookies
                    response = rq.get(url + "/leaders", cookies=cookies, params={"country": country})

                if response.status_code != 200:
                    print(f"Failed to get leaders for {country}: {response.status_code}")
                    all_leaders[country] = []
                    continue

                leaders = response.json() if isinstance(response.json(), list) else response.json().get("leaders", [])

                for leader in leaders:
                    wikipedia_url = leader.get("wikipedia_url")
                    if wikipedia_url:
                        leader['first_paragraph'] = get_first_paragraph(wikipedia_url, wiki_session)

                all_leaders[country] = leaders

            except Exception as e:
                print(f"Error for {country}: {e}")
                all_leaders[country] = [] 
        all_leaders[country] = leaders
   
    return all_leaders


Test your new functions.



In [209]:
leaders = get_leaders()

pprint(leaders)

Server is up and running!
Cookie expired for ru, fetching new cookie...
{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'first_paragraph': 'Guy Maurice Marie-Louise '
                            'Verhofstadt[5](uitspraakⓘ) (Dendermonde,11 '
                            'april1953) is eenBelgischpoliticusvoor deOpen '
                            'Vlaamse Liberalen en Democraten(Open Vld). Hij '
                            'waspremier van Belgiëvan 12 juli 1999 tot 20 '
                            'maart 2008 in drie regeringen. Hij beëindigde '
                            'zijn actieve politieke carrière in hetEuropees '
                            'Parlement, waar hij van 2009 tot 2019 '
                            'fractieleider van deAlliantie van Liberalen en '
                            'Democraten voor Europa(ALDE) was.[6]',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
 

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [210]:
# 3 lines
import json
leaders_per_country = get_leaders()
with open("leaders.json", "w", encoding="utf-8") as f:
    json.dump(leaders_per_country, f, ensure_ascii=False, indent=2) 
    """ensure_ascii=False: so that non-ASCII characters (like accents) are preserved.
    indent=2: pretty-prints with indentation for readability.
    """

Server is up and running!
Cookie expired for ru, fetching new cookie...


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [214]:
# 3 lines
# Open the file and load the dictionary
with open("leaders.json", "r", encoding="utf-8") as f:
    leaders_per_country = json.load(f)

# Check: print keys (countries)
print(leaders_per_country.keys())

dict_keys(['fr', 'be', 'ma', 'us', 'ru'])


Make a function `save(leaders_per_country)` to call this code easily.

In [215]:
# 3 lines
def save(leaders_per_country):
    with open("leaders.json", "w", encoding="utf-8") as file:
        json.dump(leaders_per_country, file, ensure_ascii=False, indent=2) 

In [216]:
# Call the function (1 line)
leaders_per_country = get_leaders()
save(leaders_per_country)

Server is up and running!
Cookie expired for ru, fetching new cookie...


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!